In [1]:
import os
import re

In [2]:
import fitz # install using: pip install PyMuPDF

In [4]:
fitz.version

('1.21.1', '1.21.1', '20221213000001')

In [6]:
print(fitz.__doc__)


PyMuPDF 1.21.1: Python bindings for the MuPDF 1.21.1 library.
Version date: 2022-12-13 00:00:01.
Built for Python 3.7 on darwin (64-bit).



In [8]:
from util.util import slice_filename

In [7]:
filename_list = os.listdir('./pdf')
filename_list

['[22.10.24]_포스트 팬데믹 시대의 통화정책 조합_신한투자증권.pdf',
 '[22.10.20]_채권 Daily (10.20)_유안타증권.pdf',
 '[22.12.08]_미국채 커브에서 덜어내려는 레벨 부담_하나증권.pdf',
 '[23.01.02]_전섹터 스프레드 축소, 등급전망 하향은 증가_현대차증권.pdf',
 '[22.12.26]_아직은 우량물에 집중할 시기_신한투자증권.pdf',
 '[22.10.12]_채권 Daily (10.12)_유안타증권.pdf',
 '[22.11.28]_기차는 아직 종작역에 도착하지 않았다_교보증권.pdf',
 '[22.09.14]_환율은 금리보다 외생변수_메리츠증권.pdf',
 '[22.10.19]_채권사이클이 돌아온다 (1)_하이투자증권.pdf',
 '[22.07.26]_한국 2분기 GDP: 민간소비가 해냈지만, 일회성에 그칠_이베스트증권.pdf',
 '[22.11.03]_속도 조절이 답은 아니다_하이투자증권.pdf',
 '[23.01.27]_채권 Daily (01.27)_유안타증권.pdf',
 '[22.12.23]_신한 Bond.Zip (12월 23일)_신한투자증권.pdf',
 '[23.01.26]_Daily Bond Morning Brief(2023.01.25)_다올투자증권.pdf',
 '[23.01.19]_BOJ 통화정책 유지 여파로 금리 급락_교보증권.pdf',
 '[23.01.19]_Eugenes FICC Update(2023.1.19)_유진투자증권.pdf',
 '[22.10.31]_채권 Daily (10.31)_유안타증권.pdf',
 '[22.08.29]_잭슨 홀 심포지엄 리뷰: 연준과 싸우지 마라 (Don’t _교보증권.pdf',
 '[23.01.05]_Daily Bond Morning Brief(2023.01.05)_다올투자증권.pdf',
 '[22.12.22]_Eugenes FICC Update(2022.12.22)_유진투자증권.pdf',
 '[22.12.12]_경계해야 할 서비스 물가 경직성_신한

In [13]:
#하나증권 Test
hana_securities_list = []
for filename in filename_list:
    slice_list = slice_filename(filename)
    company_name = slice_list[2][1:-4]
    if company_name == '하나증권':
        print(filename)
        hana_securities_list.append(filename)

[22.12.08]_미국채 커브에서 덜어내려는 레벨 부담_하나증권.pdf
[22.07.27]_성장률 서프라이즈 이후 금리 흐름_하나증권.pdf
[22.11.16]_다시 Dual Mandate_하나증권.pdf
[22.08.17]_집중호우가 물가에 미치는 영향_하나증권.pdf
[22.10.13]_10월 금통위 Review: 한은 Pivot 조건_하나증권.pdf
[22.09.07]_팔아먹고 사는 자의 슬픔_하나증권.pdf
[22.11.23]_퍼즐 맞추기_하나증권.pdf
[23.01.05]_미국 회사채 시장 Outlook: 발행(Issuance) & 만기(Ma_하나증권.pdf
[22.11.24]_11월 금통위 Review: 반환점_하나증권.pdf
[22.11.16]_진화를 넘어 진정으로_하나증권.pdf
[22.12.14]_인플레이션의 미학_하나증권.pdf
[22.08.25]_[잭슨홀 Preview] 파월의 책무와 실현 가능성_하나증권.pdf
[23.01.18]_매도자(발행사) 우위시장_하나증권.pdf
[22.10.12]_소 잃기 전에 외양간 고치기_하나증권.pdf
[22.11.08]_다시 Dual Mandate_하나증권.pdf
[23.01.06]_새롭게, 그리고 또 다시 등장한 변수들: ② 미국 대내_하나증권.pdf
[23.01.11]_낡은 연초효과_하나증권.pdf
[22.09.01]_잭슨홀 이후 무엇을 볼까_하나증권.pdf
[23.01.05]_연초효과 발생시 1분기 성과 점검_하나증권.pdf
[23.01.17]_국고5년 적정레벨 추정_하나증권.pdf
[22.09.21]_돌아온 역전의 용사_하나증권.pdf
[22.10.25]_두 마리 토끼를 잡을 수 있을까_하나증권.pdf
[22.11.11]_반환점_하나증권.pdf
[22.11.11]_좋은 친구들(Goodfellas)_하나증권.pdf
[22.09.08]_기술적 조정보다 추세 대응이 필요한 구간_하나증권.pdf
[22.08.11]_물가 피크아웃에도 갈 길 가는 연준_하나증권.pdf
[22.11.16]_장기전도 내수가 버

In [55]:
target_filename = '[23.01.05]_새롭게, 그리고 또 다시 등장한 변수들: ① 글로벌 요_하나증권.pdf'
# target_filename = '[23.01.04]_인동초(忍冬草)_하나증권.pdf'

In [52]:
def get_pdf_text(target_filename):
    text = ""
    with fitz.open('./pdf/' + target_filename) as doc:        
        for page in doc:
            text += page.get_text()

    text = text.strip()
    return text

In [56]:
with fitz.open('./pdf/' + target_filename) as doc:
    text = ""
    for page in doc:
        text += page.get_text()

text = text.strip()
print(text)

새롭게, 그리고 또 다시 등장한 변수들: ① 글로벌 요인 
Investment Strategy 
해외채권 / FX 
 
 
Global ETF / FI 박승진 
02-3771-7761 
sj81.park@hanafn.com 
 
RA 황수미 
02-3771-7721 
shwang@hanafn.com 
 
Compliance Notice 
본 조사자료는 고객의 투자에 정보를 제공할 목적으로 
작성되었으며, 어떠한 경우에도 무단 복제 및 배포 될 수 
없습니다. 또한 본 자료에 수록된 내용은 당사가 신뢰할 만한 
자료 및 정보로 얻어진 것이나, 그 정확성이나 완전성을 보장할 
수 없으므로 투자자 자신의 판단과 책임하에 최종결정을 하시기 
바랍니다. 따라서 어떠한 경우에도 본 자료는 고객의 주식투자의 
결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다. 
 
 
경기침체와 인플레이션 피크아웃 전망의 등장으로 빠르게 하락했던 미 국채금리가 
연말부터 크게 출렁이고 있다. 새로운 변수들과 기존의 변수들이 반복적으로 시장
에 반영되면서, 금리인상 사이클이 끝나면 정해진 길을 가게 될 것이라고 믿고 있던 
시장의 확신과 전망을 뒤흔들고 있기 때문이다. 연초에 박스권 흐름을 이어간 후 금
리가 레벨을 낮춰갈 것이라는 기존의 전망을 유지해야 한다는 생각이지만, 통화정
책과 시장 금리의 변화가 금융, 경제 전반에 미치는 영향이 어느 때보다 커져 있는 
시기인 만큼 여러 대내외 변수들에 대해 면밀히 점검해야 할 필요가 있겠다. 자료들
을 통해 미국 채권시장 기준의 글로벌 요인과 대내 요인을 나누어 살펴보고자 한다. 
엔화 강세: 지난 12월 FOMC가 끝난 후 진행된 통화정책회의에서, BOJ는 예상보
다 빠른 시기에 YCC 정책(변동폭 기준: ±0.50%)을 변경하였다. 기시다 총재의 
설명과는 달리, 정책 방향의 변화 인식은 일본 국채금리뿐만 아니라 환율에도 크
게 영향을 미치면서 달러대비 150엔까지 상승했던 엔화 환율이 130엔 수준까지 
빠르게 하락했다. 일본의 

In [57]:
text

'새롭게, 그리고 또 다시 등장한 변수들: ① 글로벌 요인 \nInvestment Strategy \n해외채권 / FX \n \n \nGlobal ETF / FI 박승진 \n02-3771-7761 \nsj81.park@hanafn.com \n \nRA 황수미 \n02-3771-7721 \nshwang@hanafn.com \n \nCompliance Notice \n본 조사자료는 고객의 투자에 정보를 제공할 목적으로 \n작성되었으며, 어떠한 경우에도 무단 복제 및 배포 될 수 \n없습니다. 또한 본 자료에 수록된 내용은 당사가 신뢰할 만한 \n자료 및 정보로 얻어진 것이나, 그 정확성이나 완전성을 보장할 \n수 없으므로 투자자 자신의 판단과 책임하에 최종결정을 하시기 \n바랍니다. 따라서 어떠한 경우에도 본 자료는 고객의 주식투자의 \n결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다. \n \n \n경기침체와 인플레이션 피크아웃 전망의 등장으로 빠르게 하락했던 미 국채금리가 \n연말부터 크게 출렁이고 있다. 새로운 변수들과 기존의 변수들이 반복적으로 시장\n에 반영되면서, 금리인상 사이클이 끝나면 정해진 길을 가게 될 것이라고 믿고 있던 \n시장의 확신과 전망을 뒤흔들고 있기 때문이다. 연초에 박스권 흐름을 이어간 후 금\n리가 레벨을 낮춰갈 것이라는 기존의 전망을 유지해야 한다는 생각이지만, 통화정\n책과 시장 금리의 변화가 금융, 경제 전반에 미치는 영향이 어느 때보다 커져 있는 \n시기인 만큼 여러 대내외 변수들에 대해 면밀히 점검해야 할 필요가 있겠다. 자료들\n을 통해 미국 채권시장 기준의 글로벌 요인과 대내 요인을 나누어 살펴보고자 한다. \n엔화 강세: 지난 12월 FOMC가 끝난 후 진행된 통화정책회의에서, BOJ는 예상보\n다 빠른 시기에 YCC 정책(변동폭 기준: ±0.50%)을 변경하였다. 기시다 총재의 \n설명과는 달리, 정책 방향의 변화 인식은 일본 국채금리뿐만 아니라 환율에도 크\n게 영향을 미치면서 달러대비 150엔까지 상승

In [62]:
pos = re.search('본\s+자료는\s+고객의\s+주식투자의\s+결과에\s+대한\s+법적\s+책임소재의\s+증빙자료로\s+사용될\s+수\s+없습니다.', text, re.MULTILINE | re.IGNORECASE | re.DOTALL)

In [63]:
pos

<re.Match object; span=(378, 428), match='본 자료는 고객의 주식투자의 \n결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니>

In [36]:
text[pos.end(): pos.end() + 20]

'경기침체와 인플레이션 피크아웃 전망의'

In [49]:
prev_start = 0
for pos in re.finditer(' \n \n \n', text, re.MULTILINE | re.IGNORECASE):
    section = text[prev_start: pos.start()]
    print(f"=== {prev_start} to {pos.start()-1} ===")
    print(section)
    prev_start = pos.end()

=== 0 to 62 ===
새롭게, 그리고 또 다시 등장한 변수들: ① 글로벌 요인 
Investment Strategy 
해외채권 / FX
=== 69 to 427 ===
Global ETF / FI 박승진 
02-3771-7761 
sj81.park@hanafn.com 
 
RA 황수미 
02-3771-7721 
shwang@hanafn.com 
 
Compliance Notice 
본 조사자료는 고객의 투자에 정보를 제공할 목적으로 
작성되었으며, 어떠한 경우에도 무단 복제 및 배포 될 수 
없습니다. 또한 본 자료에 수록된 내용은 당사가 신뢰할 만한 
자료 및 정보로 얻어진 것이나, 그 정확성이나 완전성을 보장할 
수 없으므로 투자자 자신의 판단과 책임하에 최종결정을 하시기 
바랍니다. 따라서 어떠한 경우에도 본 자료는 고객의 주식투자의 
결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다.
=== 434 to 2262 ===
경기침체와 인플레이션 피크아웃 전망의 등장으로 빠르게 하락했던 미 국채금리가 
연말부터 크게 출렁이고 있다. 새로운 변수들과 기존의 변수들이 반복적으로 시장
에 반영되면서, 금리인상 사이클이 끝나면 정해진 길을 가게 될 것이라고 믿고 있던 
시장의 확신과 전망을 뒤흔들고 있기 때문이다. 연초에 박스권 흐름을 이어간 후 금
리가 레벨을 낮춰갈 것이라는 기존의 전망을 유지해야 한다는 생각이지만, 통화정
책과 시장 금리의 변화가 금융, 경제 전반에 미치는 영향이 어느 때보다 커져 있는 
시기인 만큼 여러 대내외 변수들에 대해 면밀히 점검해야 할 필요가 있겠다. 자료들
을 통해 미국 채권시장 기준의 글로벌 요인과 대내 요인을 나누어 살펴보고자 한다. 
엔화 강세: 지난 12월 FOMC가 끝난 후 진행된 통화정책회의에서, BOJ는 예상보
다 빠른 시기에 YCC 정책(변동폭 기준: ±0.50%)을 변경하였다. 기시다 총재의 
설명과는 달리, 정책 방향의 변화 인식은 일본 국채금리뿐만 아니라 환율에도 크
게 영향을 미치면서 달러대비 

하나증권 보고서 분류
- 증권사 이름
- 애널리스트 이름 정보
- 커버리지


보고서 카테고리  
1. BONDView
2. CrediVille
3. Unfixed Idea
4. Credit Watch

### 하나증권 Unfixed Idea 보고서 Section 추출 테스트

In [72]:
import pdb

for target_filename in hana_securities_list:
    text = get_pdf_text(target_filename)
    
    pos = re.search('Unfixed Idea', text, re.MULTILINE | re.IGNORECASE)
    if not pos: 
        continue
    
    
    
#     for pos in re.finditer(' \n \n \n', text, re.MULTILINE | re.IGNORECASE):
#         section = text[prev_start: pos.start()]
#         # print(f"=== {prev_start} to {pos.start()-1} ===")
#         # print(section)
#         section_list.append(section)
#         prev_start = pos.end()

    pos = re.search('본\s+자료는\s+고객의\s+주식투자의\s+결과에\s+대한\s+법적\s+책임소재의\s+증빙자료로\s+사용될\s+수\s+없습니다.', text, re.MULTILINE | re.IGNORECASE | re.DOTALL)
    section = text[pos.end(): ]
    section = section.strip()
    print(f"targeting filename: {target_filename}")
    # pdb.set_trace()
    # print(section)
    sentences, text = tidy_sentences(section)
    print(sentences[-1], len(sentences), len(section))

targeting filename: [22.11.08]_다시 Dual Mandate_하나증권.pdf
시장금리 하향 안정과 헤알화의 점진적 강세가 예상된다. 24 31352
targeting filename: [23.01.06]_새롭게, 그리고 또 다시 등장한 변수들: ② 미국 대내_하나증권.pdf
다만 12월 FOMC의 점도표에서 확인할 수 있었던  것처럼, 연준 인사들의 장기 정책 판단이 크게 엇갈리고 있는 상황인 만큼, 각 경 제지표들의 영향력이 커지며 시장 변동성 확대 요인으로 작용할 전망이다. 24 5111
targeting filename: [22.11.11]_반환점_하나증권.pdf
아직 보이지 않는 인플레의 해결 돌파구로 인해 ECB의 긴축 대응이 불가피한 점 또한 저 신용등급 부도율 상향조정을 견인할 것이며 유럽 크레딧 스프레드 상승 압력은 거세질 가능성이 높다. 74 133664
targeting filename: [22.09.19]_9월 FOMC의 관전 포인트_하나증권.pdf
FOMC 이후 단기적인 변화가 나타날 수 있으나,  단기물 중심의 금리 상방 압력과 장단기 금리 역전 심화 흐름이 유지될 전망이다. 17 3286
targeting filename: [22.08.29]_예측 어려운 변수, 정직해지는 연준_하나증권.pdf
고용보고서를 확인하는 과정 에서 역전폭이 축소된다면, 포지션 재구축 기회로 활용할 필요가 있겠다. 21 4009
targeting filename: [22.09.05]_작동하기 시작하는 연준의 의도_하나증권.pdf
최근 나타난 장단기 금리차  역전폭의 되돌림 현상은 역전 확대 포지션의 재구축 기회로 활용할 필요가 있겠다. 19 4239
targeting filename: [22.08.08]_미국 고용 서프라이즈: 시간 문제이지만 경로 재조정 _하나증권.pdf
추가 상승시  10년 국채금리의 1차 상단은 2.94% 레벨에서 형성될 수 있을 것으로 판단된다. 20 4729
targeting filename: [23.01.09]

In [67]:
def tidy_sentences(section):
    # sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    # 권고, 전망 등 문장 끝맺음말 추가 필요
    sentence_enders = re.compile(r'((?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text